# Environment Sanity Check #

In [ ]:
!nvidia-smi

In [1]:
import cugraph
import cudf
import pandas as pd
import numpy as np
import networkx as nx

import plotly_express as px
%matplotlib inline

# 1. Connected Components

We first start by creating a list of edges along with the distances which we will add as the weight of the edge:


In [ ]:
edgelist = [['Mannheim', 'Frankfurt', 85], ['Mannheim', 'Karlsruhe', 80], ['Erfurt', 'Wurzburg', 186], ['Munchen', 'Numberg', 167], ['Munchen', 'Augsburg', 84], ['Munchen', 'Kassel', 502], ['Numberg', 'Stuttgart', 183], ['Numberg', 'Wurzburg', 103], ['Numberg', 'Munchen', 167], ['Stuttgart', 'Numberg', 183], ['Augsburg', 'Munchen', 84], ['Augsburg', 'Karlsruhe', 250], ['Kassel', 'Munchen', 502], ['Kassel', 'Frankfurt', 173], ['Frankfurt', 'Mannheim', 85], ['Frankfurt', 'Wurzburg', 217], ['Frankfurt', 'Kassel', 173], ['Wurzburg', 'Numberg', 103], ['Wurzburg', 'Erfurt', 186], ['Wurzburg', 'Frankfurt', 217], ['Karlsruhe', 'Mannheim', 80], ['Karlsruhe', 'Augsburg', 250],["Mumbai", "Delhi",400],["Delhi", "Kolkata",500],["Kolkata", "Bangalore",600],["TX", "NY",1200],["ALB", "NY",800]]

Now we want to find out distinct continents and their cities from this graph.
First, we will need to create a cudf dataframe with edges in it. Right now I am creating a pandas dataframe and converting it to cudf dataframe but in a real-life scenario, we will read from a csv file of edges.

In [ ]:
# create a pandas dataframe of edges
pandas_df = pd.DataFrame(edgelist)
pandas_df.columns = ['src','dst','distance']
# create a pandas dataframe of reversed edges as we have a undirected graph
rev_pandas_df = pandas_df.copy()
rev_pandas_df.columns = ['dst','src','distance']
rev_pandas_df = rev_pandas_df[['src','dst','distance']]
# concat all edges
pandas_df = pd.concat([pandas_df,rev_pandas_df])

Now our pandas df contains edges in both directions. And our node names in src and dst columns are in str format. Apparently, cuGraph doesn't like that and only works with integer node IDs.

In [ ]:
# CuGraph works with only integer node IDS
unique_destinations = set()
for [src,dst,dis] in edgelist:
  unique_destinations.add(src)
  unique_destinations.add(dst)
    
# create a map of city and a unique id
city_id_dict = {}
for i, city in enumerate(unique_destinations):
  city_id_dict[city]=i
# create 2 columns that contain the integer IDs for src and dst
pandas_df['src_int'] = pandas_df['src'].apply(lambda x : city_id_dict[x])
pandas_df['dst_int'] = pandas_df['dst'].apply(lambda x : city_id_dict[x])

Now comes the main part that we should focus on:

In [ ]:
cuda_g = cudf.DataFrame.from_pandas(pandas_df)
# cugraph needs node IDs to be int32 and weights to be float
cuda_g['src_int'] = cuda_g['src_int'].astype(np.int32)
cuda_g['dst_int'] = cuda_g['dst_int'].astype(np.int32)
cuda_g['distance'] = cuda_g['distance'].astype(np.float)
G = cugraph.Graph()
G.add_edge_list(cuda_g["src_int"],cuda_g["dst_int"] , cuda_g['distance'])
cugraph.strongly_connected_components(G)

# 2. Shortest Path

We already have our Graph as before. We can find the shortest distance from a source node to all nodes in the graph.

In [ ]:
# get distances from source node 1
distances = cugraph.sssp(G, 1)
# filter infinite distances
distances = cugraph.traversal.filter_unreachable(distances)
distances

In [ ]:
#Getting the path is as simple as:

# 1 to 15

path = []

dest = 15
while dest != 1:
   dest = distances[distances['vertex'] == dest]['predecessor'].values[0]
   path.append(dest)
print(path[::-1])

# 3. Pagerank

In [ ]:
# Loading the file as cudf
fb_cudf = cudf.read_csv("facebook_combined.txt", sep=' ', names=['src', 'dst'],dtype =['int32','int32'])

In [ ]:
# adding reverse edges also
fb_cudf = cugraph.symmetrize_df(fb_cudf, 'src', 'dst')

In [ ]:
# creating the graph
fb_G = cugraph.Graph()
fb_G.add_edge_list(fb_cudf["src"],fb_cudf["dst"])

In [ ]:
# Call cugraph.pagerank to get the pagerank scores
fb_pagerank = cugraph.pagerank(fb_G)

In [ ]:
fb_pagerank.sort_values(by='pagerank',ascending=False).head()

# 4. Link Prediction

In [ ]:
max_vertex_id = fb_pagerank['vertex'].max()

In [ ]:
max_vertex_id

In [ ]:
data = []
for x in range(0,max_vertex_id+1):
  for y in range(0,max_vertex_id+1):
    data.append([x,y])

In [ ]:
cudf_nodes =cudf.from_pandas(pd.DataFrame(data))

cudf_nodes.columns = ['src','dst']

In [ ]:
cudf_nodes['src'] = cudf_nodes['src'].astype(np.int32)
cudf_nodes['dst'] = cudf_nodes['dst'].astype(np.int32)

In [ ]:
jaccard_coeff_between_nodes = cugraph.link_prediction.jaccard(fb_G,cudf_nodes["src"],cudf_nodes["dst"])

In [ ]:
jaccard_coeff_between_nodes.head()

In [ ]:
len(jaccard_coeff_between_nodes)

In [ ]:
jaccard_coeff_between_nodes=jaccard_coeff_between_nodes[jaccard_coeff_between_nodes['source']!=jaccard_coeff_between_nodes['destination']]

In [ ]:
fb_cudf.columns = ['source',	'destination']

In [ ]:
fb_cudf['edgeflag']=1

In [ ]:
jaccard_coeff_joined_with_edges = jaccard_coeff_between_nodes.merge(fb_cudf,on= ['source',	'destination'],how='left')


In [ ]:
jaccard_coeff_joined_with_edges.head()

In [ ]:
# We just want to see the jaccard coeff of new edges
new_edges_jaccard_coeff = jaccard_coeff_joined_with_edges[jaccard_coeff_joined_with_edges['edgeflag']!=1]

In [ ]:
# Here are the predicted edges from our metric.
new_edges_jaccard_coeff.sort_values(by='jaccard_coeff',ascending=False).head(50)

# 5. Basic Measures

In [ ]:
print("Number of Nodes",fb_G.number_of_nodes())
print("Number of Edges",fb_G.number_of_edges())

In [ ]:
# Compute the indegree and outdegree to the node. In a directed graph this corresponds to no of followers and no of follows
fb_G.degrees().head()

# Benchmarking

## A. On Facebook Data

In [ ]:
# Creating graphs for Benchmarking using cuDF and networkx
# graphX
# Loading the file as cudf

fb_cudf = cudf.read_csv("facebook_combined.txt", sep=' ', names=['src', 'dst'],dtype =['int32','int32'])

In [ ]:
fb_cudf = cugraph.symmetrize_df(fb_cudf, 'src', 'dst')

In [ ]:
fb_G = cugraph.Graph()
fb_G.add_edge_list(fb_cudf["src"],fb_cudf["dst"])

In [ ]:
# reading the dataset
fb = nx.read_edgelist('facebook_combined.txt', create_using = nx.Graph(), nodetype = int)

### 1. Connected Components

In [ ]:
%%timeit
ccs = []
for i, x in enumerate(nx.connected_components(fb)):
    ccs.append(x)

In [ ]:
%%timeit
ccs = cugraph.weakly_connected_components(fb_G)

### 2. Shortest Path

In [ ]:
%%timeit
nx.single_source_shortest_path(fb,1)

In [ ]:
%%timeit
distances = cugraph.sssp(fb_G, 1)

### 3. Pagerank

In [ ]:
%%timeit
pageranks = nx.pagerank(fb)

In [ ]:
%%timeit
fb_pagerank = cugraph.pagerank(fb_G)

## B. On Twitter Data

In [2]:
# Creating graphs for Benchmarking using cuDF and networkx
# graphX
# Loading the file as cudf

twitter_cudf = cudf.read_csv("twitter_combined.txt", sep=' ', names=['src', 'dst'],dtype =['int32','int32'])

In [3]:
twitter_cudf = cugraph.symmetrize_df(twitter_cudf, 'src', 'dst')

In [4]:
twitter_cudf['src_int'], twitter_cudf['dst_int'], number = cugraph.renumber( twitter_cudf['src'], twitter_cudf['dst'] )

In [5]:
twitter_G = cugraph.Graph()
twitter_G.add_edge_list(twitter_cudf["src_int"],twitter_cudf["dst_int"])

In [ ]:
# reading the dataset
twitter = nx.read_edgelist('twitter_combined.txt', create_using = nx.Graph(), nodetype = int)

### 1. Connected Components

In [ ]:
%%timeit
ccs = []
for i, x in enumerate(nx.connected_components(twitter)):
    ccs.append(x)

In [6]:
%%timeit
ccs = cugraph.weakly_connected_components(twitter_G)

33.9 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 2. Shortest Paths

In [ ]:
%%timeit
nx.single_source_shortest_path(twitter,35389442)

In [ ]:
%%timeit
distances = cugraph.sssp(twitter_G, 1)

### 3. Pagerank

In [ ]:
%%time
pageranks = nx.pagerank(fb)

In [ ]:
%%timeit
fb_pagerank = cugraph.pagerank(fb_G)

## C. Google Plus Data

In [7]:
# reading the dataset
gplus = nx.read_edgelist('gplus_combined.txt', create_using = nx.Graph(), nodetype = int)

In [8]:
pd_gplus_df = pd.read_csv("gplus_combined.txt", sep=' ', names=['src', 'dst'])

nodes = set(list(pd_gplus_df['src'].unique()) + list(pd_gplus_df['dst'].unique()))

nodes_int_dict = {}
for i,node in enumerate(nodes):
    nodes_int_dict[node]=i
    

pd_gplus_df['src_int'] = pd_gplus_df['src'].apply(lambda x : nodes_int_dict[x])
pd_gplus_df['dst_int'] = pd_gplus_df['dst'].apply(lambda x : nodes_int_dict[x])

In [9]:
gplus_cudf = cudf.from_pandas(pd_gplus_df)
gplus_cudf['src_int'] = gplus_cudf['src_int'].astype(np.int32)
gplus_cudf['dst_int'] = gplus_cudf['dst_int'].astype(np.int32)

In [10]:
gplus_cudf = cugraph.symmetrize_df(gplus_cudf, 'src_int', 'dst_int')

In [11]:
gplus_G = cugraph.Graph()
gplus_G.add_edge_list(gplus_cudf["src_int"],gplus_cudf["dst_int"])

### 1. Connected Components

In [ ]:
%%time
ccs = []
for i, x in enumerate(nx.connected_components(gplus)):
    ccs.append(x)

In [12]:
%%time
ccs = cugraph.weakly_connected_components(gplus_G)

CPU times: user 1.74 s, sys: 668 ms, total: 2.41 s
Wall time: 2.4 s


### 2. Shortest Paths

In [ ]:
%%time
ssp = nx.single_source_shortest_path(gplus,109247306373593947755)

In [ ]:
%%time
ssp = cugraph.sssp(gplus_G, 0)

### 3. Pagerank

In [ ]:
%%time
pageranks = nx.pagerank(gplus)

In [ ]:
%%time
fb_pagerank = cugraph.pagerank(gplus_G)

## Results

Connected Components

In [ ]:
results = pd.read_excel("results.xlsx")

In [ ]:
results.head()

In [ ]:
px.bar(results,x ='Dataset',y='Time',color = 'Module',facet_row = 'Algorithm')